# Data Collection: Pokémon and News Sites

Name: `Andrea Jean C. Marcelo`<br>
Partner: `Francheska Vicente`- recommended the news site, Philipine Daily Inquirer <br>
Section: `N01`



### Importing libraries

In [1]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup

# **1 Bulbapedia**

**Output:** List of the `next generations of Pokemons` with the following information:
- kdex
- ndex
- name
- types
- URL to the Pokemon's wiki page

### 1.1 Loading the `Bulbapedia` page

In [2]:
URL_bulba="https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"
bulba_page=requests.get(URL_bulba)
bulba_page

<Response [200]>

In [3]:
bulba_soup = BeautifulSoup(bulba_page.content, 'html.parser')
bulba_soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of Pokémon by National Pokédex number - Bulbapedia, the community-driven Pokémon encyclopedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"06369d940b7579534d49094c","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Pokémon_by_National_Pokédex_number","wgTitle":"List of Pokémon by National Pokédex number","wgCurRevisionId":3498912,"wgRevisionId":3498912,"wgArticleId":65356,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Lists of Pokémon","Lists"],"wgPageContentLanguage":"en"

In [4]:
# Get main content <div>
bulba_content=bulba_soup.find(id='mw-content-text')

# Get all <table> elements
bulba_tables=bulba_content.find_all('table')
len(bulba_tables)

13

### 1.2 Getting the list of the `next generations of Pokemons` 

In [5]:
gen_lists = []
for gen in range(2,10):
    gen_lists.append(bulba_tables[gen])
len(gen_lists)

8

### 1.3 Getting and organizing the `needed details for each pokemon generations list`

In [6]:
gen_lists_json = []

for gen in range(0,8):
    # getting the pokemon infos
    per_gen_json = []
    gen_row=gen_lists[gen].contents[1]
    
    for pokemon_info_values, even_index_chec in zip(gen_row.contents, range(0,len(gen_row.contents))):
        # Pokemons' values are stored in even index (divisible by 2 and is not 0)
        if ((even_index_chec % 2) == 0) & (even_index_chec != 0) :
            url = 'https://bulbapedia.bulbagarden.net' + gen_row.contents[even_index_chec].find_all('a')[0]['href']
            pokemon_raw_info = pokemon_info_values.text.strip().split('\n')   
            
            ## Pokemons that are other regional forms and has 1 type
            if len(pokemon_raw_info) == 7:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[1]
                name = pokemon_raw_info[4]
                type1 = pokemon_raw_info[6]
                type2 = ''
               
            ## Pokemons that are other regional forms and has 2 types
            elif len(pokemon_raw_info) == 8:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[1]
                name = pokemon_raw_info[4]
                type1 = pokemon_raw_info[6]
                type2 = pokemon_raw_info[7]
                
            ## Orig pokemon list with single typing
            elif len(pokemon_raw_info) == 9:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[2]
                name = pokemon_raw_info[6]
                type1 = pokemon_raw_info[8]
                type2 = ''
                
            # Orig pokemon list with multiple typing
            elif len(pokemon_raw_info) == 10:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[2]
                name = pokemon_raw_info[6]
                type1 = pokemon_raw_info[8]
                type2 = pokemon_raw_info[9]
                 
            else:
                print('Handle pokemons with ' + str(len(pokemon_raw_info)) + ' elements details')

            # Saving as a tuple
            per_gen_json.append({"kdex" : kdex,
                              "ndex" : ndex,
                              "name" : name,
                              "type1": type1,
                                "type2": type2,
                                "URL to the Pokemon's wiki page" : url})
    n = gen+2
    generation = "Generation " + str(n)
    gen_lists_json.append({generation : per_gen_json})

### 1.4 Saving the list of generations of pokemon in `json`

In [7]:
with open('Bulbapedia.json', 'w') as outfile:
    json.dump(gen_lists_json, outfile)

# **2. News site: Philippine Daily Inquirer**

**Output:** List of `news articles published on April 4 - 5` with the following information:
- date
- title
- full article
- author

### 2.1 Loading the `Philippine Daily Inquirer` page and Gathering the list of `articles released from April 4, 2022 to April 5, 2022`

In [8]:
date_range = ['April 04, 2022', 'April 05, 2022']
flag = False
page_ctr = 1
news_list = []

while not flag:
    # Requesting an Inquirer news page
    URL_news="https://newsinfo.inquirer.net/source/philippine-daily-inquirer"
    if page_ctr > 1:
        URL_news += ("/page/" + str(page_ctr))
    print('Reached Page ' + str(page_ctr) + ' ' + URL_news)
    news_page=requests.get(URL_news)
    
    # Parse HTML data
    news_soup = BeautifulSoup(news_page.content, 'html.parser')
    
    # Accessing data
    news_content= news_soup.find(id='inq-channel-left')
    news=news_content.find_all(id='ch-ls-box')
    
    # Access each article
    for n in range(len(news)):
        if n == 4 or n == 9:
            continue
        else:
            if news[n]('span')[0].text in date_range:
                news_list.append(news[n])
            if news[n]('span')[0].text == 'April 03, 2022':
                flag = True
    page_ctr+=1 

Reached Page 1 https://newsinfo.inquirer.net/source/philippine-daily-inquirer
Reached Page 2 https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/2
Reached Page 3 https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/3
Reached Page 4 https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/4
Reached Page 5 https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/5
Reached Page 6 https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/6
Reached Page 7 https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/7
Reached Page 8 https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/8
Reached Page 9 https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/9
Reached Page 10 https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/10
Reached Page 11 https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/11
Reached Page 12 https://newsinfo.inquirer.net/source/philippine-dail

### 2.2 Checking the `gathered articles` and the pattern of their data

In [9]:

n = 0
for each in news_list:
    n+=1
    print(each)
    print('************************** '+ str(n)+' **************************')

<div id="ch-ls-box">
<a href="https://newsinfo.inquirer.net/1578299/polls-must-not-delay-subsidy-for-drivers-senators">
<div id="ch-ls-img" style="background-image: url(https://newsinfo.inquirer.net/wp-content/blogs.dir/10/files/2022/04/274223.jpeg)">
</div>
</a>
<div id="ch-ls-head">
<div id="ch-cat">Headlines</div>
<h2><a href="https://newsinfo.inquirer.net/1578299/polls-must-not-delay-subsidy-for-drivers-senators">Polls must not delay subsidy for drivers – senators</a></h2>
<div id="ch-postdate"><span>April 05, 2022</span> <span>BY:  DJ Yap, Dona Z. Pazzibugan, Krixia Subingsubing</span></div>
</div>
</div>
************************** 1 **************************
<div id="ch-ls-box">
<a href="https://newsinfo.inquirer.net/1578310/govt-eyes-new-definition-of-full-vaccination">
<div id="ch-ls-img" style="background-image: url(https://newsinfo.inquirer.net/wp-content/blogs.dir/10/files/2021/04/News140934.jpg)">
</div>
</a>
<div id="ch-ls-head">
<div id="ch-cat">Headlines</div>
<h2><a hr

### 2.3 Getting the `needed details for each article`

In [10]:
news_json = []
for article_ctr in range(0,len(news_list)):
    # Getting the date, title, author
    details = news_list[article_ctr].contents[3].text.strip().split('\n')
    date_author = details[2].split('BY:\xa0')
    date_author.append('')
    # Getting the full article
    url = news_list[article_ctr].contents[3].find('a')['href']
    article_page=requests.get(url)
    article_soup = BeautifulSoup(article_page.content, 'html.parser')
    article_content=article_soup.find(id='article_content')
    article_texts = article_content.text.strip().split('\n')
    full_article = ''
    for each in article_texts:
        if each == '': 
            continue
        else:
            full_article+=(each + ' ')
    # Saving as a tuple
    news_json.append({"date" : date_author[0].strip(),
                        "title" : details[1],
                          "full_article" : full_article.strip(),
                          "author" : date_author[1].strip()})

### 2.4 Saving the list of news articles released from April 4, 2022 to April 5, 2022 in `json`

In [11]:
with open('News.json', 'w') as outfile:
    json.dump(news_json, outfile)